QAOA for Max-Cut — Learning Qiskit

Here I use Qiskit to explore how the Quantum Approximate
Optimization Algorithm (QAOA) works on a Max-Cut problem

The basic idea:
- We have a set of points (nodes) connected by lines (edges)
- We want to split the points into two groups so that as many lines as possible
  go between the groups
- The number of “between-group” lines is called the cut size which is what will be maximized

Why QAOA?
- QAOA is a hybrid quantum–classical algorithm for solving optimization problems
- The quantum circuit generates solutions
- A classical loop adjusts the circuit parameters for optimization

Plan for this notebook:
1. Make a small random graph
2. Build a simple QAOA circuit for it
3. Run the circuit on Qiskit’s Aer simulator to get measurement results
4. Score the results by their cut size
5. Compare to the exact best cut for the same graph
6. Plot the best split we find

Goal: Understand the basics of building, running, and analyzing a quantum circuit in Qiskit


In [3]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from scipy.optimize import minimize

from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer

In [ ]:
# helper functions 

def cut_value(G, bitstring: str) -> int:
    """
    Score a bitstring ("0101") for Max-Cut
    +1 for every edge whose endpoints land in different groups (0 vs 1)
    """
    z = np.array([1 if c == '1' else 0 for c in bitstring], dtype=int) # stores as array 
    return sum(int(z[u] != z[v]) for u, v in G.edges()) # summing the nodes in each edge if different group

def expected_cut_from_counts(G, counts: dict) -> float:
    """
    Turn raw measurement counts into an expected cut size.
    (Weighted average of cut_value over all observed bitstrings)
    """
    shots = sum(counts.values()) # times the circut was ran 
    return sum((freq / shots) * cut_value(G, s) for s, freq in counts.items()) # probability * cut edges so it's like a weighting factor 

def qaoa_circuit_p1(G, gamma: float, beta: float) -> QuantumCircuit:
    """
    Build a p=1 QAOA circuit for Max-Cut (1 cost layer -> 1 mixer layer -> measure)
    - Start in uniform superposition (H on all qubits)
    - Cost layer per edge: CX(i,j) -> RZ(2γ) on j -> CX(i,j)
    - Mixer: RX(2β) on each qubit
    - Measure all qubits
    """
    # start in uniform superposition by applying Hadamard gate (putting the qubit into the sp)
    n = G.number_of_nodes() # nodes in graph
    qc = QuantumCircuit(n, n) # create qc with n qubits and n classical to store measurement 
    qc.h(range(n)) # apply H gate for equal super pos 

    # cost layer for 1 edge(i,j), adds phase for max-cut
    for i, j in G.edges():
        qc.cx(i, j) # ties bits together so z-rotation depends on i and j
        qc.rz(2 * gamma, j) # zz-rotation that adds phase based off of parity 
        qc.cx(i, j) # unties to restore structure of circuit 

    # mixer layer for phase difference interference so prob is towards better cuts 
    for q in range(n):
        qc.rx(2 * beta, q) # RX rotation by angle 2*beta on qubit q for prob to be either 0 or 1

    qc.measure(range(n), range(n)) # actually measure and store as 0 or 1

    return qc
